# DPA3: A Graph Neural Network for the Era of Large Atomistic Models

## Introduction

**DPA3** is a multi-layer graph neural network founded on line graph series (LiGS), designed explicitly for the era of [large atomistic models (LAMs)](https://www.aissquare.com/openlam). The generalization error of the DPA3 model adheres to the scaling law. The scalability in the number of model parameters is attained by stacking additional layers within DPA3. Additionally, the model employs a dataset encoding mechanism that decouples the scaling of training data size from the model size within its multi-task training framework.

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/8ca7545d-d0f6-4952-9d6a-fc615a950f9e.png" alt="" width="600" style="display: block; margin: auto;">

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/c5adcd1b-0721-400b-a7c9-c8e4f25a78ec.png" alt="" width="600" style="display: block; margin: auto;">

When trained as problem-oriented potential energy models, the DPA3 model exhibits superior accuracy in the majority of benchmark cases, encompassing systems with diverse features, including molecules, bulk materials, surface and cluster catalysts, two-dimensional materials, and battery materials. When trained as a LAM on the OpenLAM-v1 dataset, the DPA-3.1-3M model exhibits state-of-the-art performance in the LAMBench benchmark suite for LAMs, demonstrating lowest overall zero-shot generalization error across 17 downstream tasks from a broad spectrum of research domains. This performance suggests superior accuracy as an out-of-the-box potential model, requiring minimal fine-tuning data for downstream scientific applications.

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/79ee197d-4283-4403-b3fe-5b0758e992a8.png" alt="" width="600" style="display: block; margin: auto;">



<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/0ab27a7f-a35b-463c-8606-e740a8d0d328.png" alt="" width="600" style="display: block; margin: auto;">


Reference: 

[1] Zhang, D., Peng, A., Cai, C., Li, W., Zhou, Y., Zeng, J., ... & Wang, H. (2025). Graph neural network model for the era of large atomistic models. [arXiv preprint arXiv:2506.01686](https://arxiv.org/abs/2506.01686).


**This notebook will detail the steps to train DPA3 model from scratch, finetune a pretrained model, perfrom python inference and run MD using ASE calculator.** 

**Note that this notebook will mainly focus on DPA3 model training and finetuning. For data preparation and other basic usages of DeePMD-kit, please refer to [this notebook](https://bohrium.dp.tech/notebooks/71979198691) first.**

## 1. Installation

We install DeePMD-kit v3.1.0 to run DPA3.

In [ ]:
!git clone https://github.com/deepmodeling/deepmd-kit.git -b v3.1.0rc0

In [ ]:
!cd deepmd-kit && pip install -e .

In [ ]:
!dp --version

## 2. Train the DPA3 model from scratch

We will first train a 6-layer DPA3 model from scratch on the example water system.

In [ ]:
!cd /deepmd-kit/examples/water/dpa3 && cat input_torch.json

The definitions of params in DPA3 desciptor can be found [here](https://docs.deepmodeling.com/projects/deepmd/en/v3.1.0rc0/train/train-input.html#argument:model[standard]/descriptor[dpa3]/repflow).

For demonstration simplicity, **we will use only 200 training steps in this example**:

In [ ]:
!cd /deepmd-kit/examples/water/dpa3 && sed -i "s/\"numb_steps\": 1000000/\"numb_steps\": 200/g" input_torch.json

Note that optimal performance typically requires around **1 million steps or more** in real-world scenarios.

We then run training:

In [ ]:
!cd /deepmd-kit/examples/water/dpa3 && dp --pt train input_torch.json --skip-neighbor-stat

Now we get a toy DPA3 model for just demonstration.

## 3. Finetune from the pretrained DPA3 model

Next, we provide DPA3 models pretrained on [**OpenLAM-v1 datasets**](https://www.aissquare.com/datasets/detail?pageType=datasets&name=OpenLAM-TrainingSet-v1&id=308), including application-oriented datasets such as Alloy, Semiconductor, and Drug-like molecules, as well as open source DFT datasets like OMat, MPtraj, and OC20.

First we copy two pretrained DPA3 models (3 layers and 9 layers) from mounted Bohrium dataset:

In [ ]:
!cp -r /bohr/dpa3-model-d4ju/v2 ./dpa3-model && tree ./dpa3-model -L 1

This directory contains the following main files:
1. **DPA-3.1-3M.pt** : multi-task pretrained DPA3-L16 model on OpenLAM-v1 datasets, downloaded from [AIS-Square](https://www.aissquare.com/models/detail?pageType=models&name=DPA-3.1-3M&id=343).
2. **input_pretrain_3.1_3M.json** : the corresponding pretrained input for **DPA-3.1-3M.pt**.

3. Other files in preview edition: **0211_prod_9wide_300w.pt**, **0217_l3_300w.pt** are different DPA3 variants also pre-trained on OpenLAM-v1 datasets, with corresponding inputs **9wide_input.json** and **3wide_input.json**.

We will demonstrate fine-tuning using **DPA-3.1-3M.pt** as an example. 
Readers can also apply identical procedures to other variants.

First we will prepare the input for finetuning:

In [ ]:
!cd /deepmd-kit/examples/water && mkdir -p dpa3_finetune && cp /dpa3-model/DPA-3.1-3M.pt ./dpa3_finetune && cp /dpa3-model/input_pretrain_3.1_3M.json ./dpa3_finetune && cp dpa3/input_torch.json ./dpa3_finetune && tree /deepmd-kit/examples/water/dpa3_finetune -L 1

When performing fine-tuning based on pretrained models, it is crucial to **maintain identical model configurations (`"model"` scope in json file) between the pretraining and fine-tuning phases**. 

The model configuration may differ from our previous from-scratch training examples, therefore we will perform the following replacement to ensure the same model section in the finetune input:

In [ ]:
import json

## Load pretrained model input
pretrained_input = json.load(open('/deepmd-kit/examples/water/dpa3_finetune/input_pretrain_3.1_3M.json', 'r'))

## Load example input
example_input = json.load(open('/deepmd-kit/examples/water/dpa3_finetune/input_torch.json', 'r'))

## Get pretrained model configuration
dpa3_descriptor = pretrained_input['model']['shared_dict']['dpa3_descriptor']
fitting = pretrained_input['model']['shared_dict']['shared_fitting']

## Do model configuration replacement
example_input['model']['descriptor'] = dpa3_descriptor
example_input['model']['fitting_net'] = fitting

# (Optional) set a smaller start learning rate
example_input['learning_rate']['start_lr'] = 1e-4

## Save finetune input
json.dump(example_input, open('/deepmd-kit/examples/water/dpa3_finetune/input_finetune.json', 'w'), indent=4)

In [ ]:
!cd /deepmd-kit/examples/water/dpa3_finetune && cat input_finetune.json

Then, note that the pretrained model is pretrained in a multi-task manner, featuring a unified backbone (referred to as the unified descriptor) and several fitting nets for different datasets. For detailed information, refer to the [DPA2 paper](https://www.nature.com/articles/s41524-024-01493-2).

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/b46229d6746e4358be3181363a158efb/94935036-1adb-4562-ae45-b7839e8666ae.png)

Before finetuning, we need to **select a specific fitting net** from the model to make predictions. To list the available fitting nets (model-branch) in this pretrained model, use the following command:

In [ ]:
!cd /deepmd-kit/examples/water/dpa3_finetune && dp --pt show DPA-3.1-3M.pt model-branch

This lists all the available fitting nets. Then you need to select a fitting net that closely matches your system, here we take `MP_traj_v024_alldata_mixu`, which stands for branch of MPtraj dataset for example.

In [ ]:
!cd /deepmd-kit/examples/water/dpa3_finetune && dp --pt train input_finetune.json --skip-neighbor-stat --finetune DPA-3.1-3M.pt --model-branch MP_traj_v024_alldata_mixu

Then finally we have a finetuned DPA3 model.

## 4. Test models

After completing training/finetuning, we typically evaluate model performance by examining accuracy across different material systems.

First we can compare the loss curves (lcurve.out) during training/finetuning:

In [ ]:
!head /deepmd-kit/examples/water/dpa3/lcurve.out

In [ ]:
!head /deepmd-kit/examples/water/dpa3_finetune/lcurve.out

Then we will typically run inference directly on target systems using the `dp test` command to measure prediction errors, which provides more precise accuracy evaluation:

In [ ]:
!dp test --help

In [ ]:
!dp --pt test -m /deepmd-kit/examples/water/dpa3/model.ckpt.pt -s /deepmd-kit/examples/water/data/data_3

In [ ]:
!dp --pt test -m /deepmd-kit/examples/water/dpa3_finetune/model.ckpt.pt -s /deepmd-kit/examples/water/data/data_3

We can see that the fine-tuned model demonstrates significantly better accuracy (benefiting from the pre-trained model).

## 5. Python interface for MD

Finally, we demonstrate how to use trained models for **practical simulations in Python via the ASE interface**. 

In [ ]:
!pip install ase

We need first freeze out checkout file into one `pth` frozen model:

In [ ]:
!cd /deepmd-kit/examples/water/dpa3_finetune && dp --pt freeze -o dpa3_water_finetune.pth

We can also directly freeze the pretrained model without further finetuning to perform MD: 

```
dp --pt freeze -c DPA-3.1-3M.pt -o dpa3_water_branch.pth --model-branch H2O_H2O_PD

```



Then we can run the following Python code for prediction or optimization:

In [ ]:
%%writefile ase_md.py
from ase import Atoms
from deepmd.calculator import DP

## Compute potential energy
dp = DP("/deepmd-kit/examples/water/dpa3_finetune/dpa3_water_finetune.pth")
water = Atoms('H2O', positions=[(0.7601, 1.9270, 1), (1.9575, 1, 1), (1., 1., 1.)], cell=[100, 100, 100])
water.calc = dp
print(water.get_potential_energy())
print(water.get_forces())

## Run BFGS structure optimization
from ase.optimize import BFGS
dyn = BFGS(water)
dyn.run(fmax=1e-3)
print(water.get_positions())

In [ ]:
!python ase_md.py

### Ref
1. [DPA3 paper](https://arxiv.org/abs/2506.01686)
2. [LAMBench paper](https://arxiv.org/abs/2504.19578)
2. [DPA2 paper](https://www.nature.com/articles/s41524-024-01278-7)
2. [DPA2 finetune tutorial](https://www.aissquare.com/models/detail?pageType=models&name=DPA-2.3.1-v3.0.0rc0&id=287)
3. [AIS-Square](https://www.aissquare.com)
4. [OpenLAM Initiative](https://www.aissquare.com/openlam)
5. [LAMBench Website](https://www.aissquare.com/openlam?tab=Benchmark)